### Objective

In this notebook, we attempt to build surrogate models for predicting the thermal resistance.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys

from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import xgboost as xgb
import sklearn.gaussian_process as gp
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, max_error

# Custom Gaussian Process model
GP_path = os.path.abspath(os.path.join('../../'))
if GP_path not in sys.path:
    sys.path.append(GP_path+"\\GaussianProcess")
from GPInterpolator import GPInterpolator

### 1. Load dataset

In [2]:
df = pd.read_csv('./Dataset/TcTj_train.csv', header=None)
df.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']
print(f"Pool: {df.shape[0]}")

Pool: 9421


In [7]:
df.iloc[0, :-3].to_list()

[397.98,
 381.19,
 0.021736,
 0.20004,
 0.32107,
 0.015976,
 0.045993,
 45.0,
 0.0011509,
 0.062429,
 0.12079,
 0.12857,
 0.059733]

In [ ]:
# Remove outliers
df = df[df.Tj<250].reset_index(drop=True)
print(f"Filtered training pool: {df.shape[0]}")

In [ ]:
# Dedicated testing set
df_test = pd.read_csv('./Dataset/TcTj_test.csv', header=None)
df_test.columns = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2', 'Tc', 'Tj', 'w']

# Remove outliers
df_test = df_test[df_test.Tj<250].reset_index(drop=True)
print(f"Filtered testing pol: {df_test.shape[0]}")

In [ ]:
def create_samples(df, train_num):
   
    # Create dataset
    X = df.iloc[:, :-3].to_numpy()
    y = df.iloc[:, -2].to_numpy()
    
    # Train-test split
    if train_num < len(df):
        test_size = 1-train_num/len(df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    else:
        X_train, y_train = X, y
        X_test, y_test = None, None
    
    return X_train, X_test, y_train, y_test

### 2. Model training

#### 2.1 Model specifications

In [ ]:
def model_fit(model_type, X_train, y_train, hyperparams, GP_model=None):
    """This function is used for training ML models."""
    
    # Construct pipeline
    if model_type == 'Gaussian Process':
        # Custom GP
        model = Pipeline([
            ('scaler', MinMaxScaler()),
            ('regressor', GPInterpolator(**hyperparams[model_type]))
        ])        
        
    elif model_type == 'Gaussian Process (sklearn)':
        model = Pipeline([
            ('scaler', MinMaxScaler()),
            ('regressor', gp.GaussianProcessRegressor(**hyperparams[model_type]))
        ])

    elif model_type == 'XGBoost':
        model = xgb.XGBRegressor(**hyperparams[model_type])
    
    else:
        raise KeyError('Unrecognized model type!')
    
    # Fit the pipeline
    model.fit(X_train, y_train)
    
    return model

#### 2.2 ML training

In [ ]:
def evaluate_model(y_true, y_pred):
    """This function is used for evaluating the ML models performance."""
    
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    max_e = max_error(y_true, y_pred)
    
    percentage = np.abs(y_true-y_pred)/y_true
    max_percentage = np.max(percentage)*100
    max_percentage_loc = np.argmax(percentage)
    mean_percentage = np.mean(percentage)*100
    
    return rmse, max_e, max_percentage, max_percentage_loc, mean_percentage

In [ ]:
model_hyperparams = {
    'XGBoost': {'n_estimators': 2000, 'max_depth': 5, 'learning_rate': 0.05, 'reg_lambda': 10,
               'gamma': 0, 'subsample': 0.3, 'colsample_bytree': 1, 'random_state': 10, 'n_jobs': -1},
    'Gaussian Process': {
        'n_restarts': 20,
        'kernel': 'Gaussian',
        'trend': 'Const',
        'opt': {'optimizer':'L-BFGS-B', 'jac': True}
    },
    'Gaussian Process (sklearn)': {
        'kernel': gp.kernels.ConstantKernel(1.0, (1e-3, 1e3)) * gp.kernels.RBF(1.0, (1e-3, 1e3)),
        'optimizer': 'fmin_l_bfgs_b',
        'n_restarts_optimizer': 10,
        'alpha': 1e-10,
        'normalize_y': True,
        'random_state': 10
    }
}

In [ ]:
# Train-test split
X_train, _, y_train, _ = create_samples(df, 9000)
X_test, _, y_test, _ = create_samples(df_test, 9000)

# Fit the model
model_type = 'XGBoost'
model = model_fit(model_type, X_train, y_train, model_hyperparams)

In [ ]:
# Assess the model
if model_type == 'Gaussian Process':
    y_pred, _ = model.predict(X_test)

else:
    y_pred = model.predict(X_test)
    
rmse, max_e, max_per, _, mean_per = evaluate_model(y_test, y_pred)
print(f"RMSE: {rmse:.4f} / data std: {np.std(y_test):.4f}")
print(f"Max Error: {max_e:.4f}")
print(f"Max Percentage Error: {max_per:.2f}")
print(f"Mean Percentage Error: {mean_per:.2f}")

In [ ]:
# Set the default font size
plt.rcParams['font.size'] = 14

fig, ax = plt.subplots(figsize=(5, 4))
ax.plot(y_test, y_pred, 'o')
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
ax.set_title('Tjmax Calculation')
ax.set_xlabel('Ground truth')
ax.set_ylabel('Prediction')

plt.tight_layout()

#### 2.3 Sensitivity analysis

#### XGBoost feature importance 

In [ ]:
model.get_booster().feature_names = ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2']
feature_importance = model.get_booster().get_score(importance_type='gain')

# Plotting feature importance
xgb.plot_importance(model)
plt.show();

#### Sobol indices

In [ ]:
from SALib.sample import sobol as sobol_sample
from SALib.analyze import sobol

In [ ]:
def check_constraints(X):

    # Dimension
    c_module, d_module = 61.4e-3, 106e-3
    
    # Position
    Xc_min = c_module / 2
    Xc_max = X[3] - c_module / 2
    Yc_min = d_module / 2
    Yc_max = X[4] - d_module / 2
    
    con1 = X[8] < X[3] / X[7] - 1e-3     # For t
    con2 = (X[-4] < Xc_max) and (X[-4] > Xc_min) and (X[-3] < Yc_max) and (X[-3] > Yc_min)
    con3 = (X[-2] < Xc_max) and (X[-2] > Xc_min) and (X[-1] < Yc_max) and (X[-1] > Yc_min)
    con4 = (np.abs(X[-4] - X[-2]) > c_module) | (np.abs(X[-3] - X[-1]) > d_module)
    return con1 and con2 and con3 and con4

In [ ]:
bounds = []
for feature in df.columns.to_list()[:-3]:
    bounds.append([df[feature].to_numpy().min(), df[feature].to_numpy().max()])

# Define your problem with the input variables
problem = {
    'num_vars': 13,  # Adjust the number of variables based on your model
    'names': ['Q1', 'Q2', 'd', 'b', 'L', 'c', 'L_duct', 'n', 't', 'xc1', 'yc1', 'xc2', 'yc2'],  # Names of the variables
    'bounds': bounds # Bounds for each variable, adjust accordingly
}

# Generate samples using the Saltelli sampler
N = 16384

# Check sensitivity indices
index_type = "ST"

In [ ]:
%%time

def evaluate_model(X):
    predictions = model.predict(X)
    return predictions

# Generate samples as before
samples = sobol_sample.sample(problem, N)

# Switch Q1 & Q2 (Q1 always larger than Q2)
swap = samples[:, 0] < samples[:, 1]
samples[swap, 0], samples[swap, 1] = samples[swap, 1], samples[swap, 0].copy()

# Swap locations as well
samples[swap, -4], samples[swap, -2] = samples[swap, -2], samples[swap, -4].copy()
samples[swap, -3], samples[swap, -1] = samples[swap, -1], samples[swap, -3].copy()

# Rejection sampling
valid_samples = np.array([s for s in samples if check_constraints(s)])

# Evaluate the model using the XGBoost model
expected_Y_size = (2*samples.shape[1]+2)
num_full_sets = len(valid_samples) // expected_Y_size
adjusted_valid_samples = valid_samples[:num_full_sets * expected_Y_size]
Y = evaluate_model(adjusted_valid_samples)
print(f"True sample size: {len(Y)}/{N}")

# Proceed with Sobol analysis as before
sobol_indices = sobol.analyze(problem, Y)
sobol_indices = sobol_indices[index_type]
variable_names = problem['names']

In [ ]:
# Extract and print first-order Sobol indices
indices_with_names = list(zip(sobol_indices, variable_names))
indices_with_names.sort(reverse=True)
sorted_indices, sorted_names = zip(*indices_with_names)

# Creating the plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_indices)), sorted_indices, tick_label=sorted_names)
plt.xlabel('Sobol Index')
plt.title('Feature Sensitivity Ranking: Tjmax')
plt.gca().invert_yaxis()  # Invert y-axis to have the most sensitive feature on top
plt.show()

#### Weight sensitivity

In [ ]:
%%time

def evaluate_weight(X):
    # Properties
    density_Al = 2700
    Fan_height = 40e-3
    Fan_Weight = 50.8e-3
    N_fan = np.ceil(X[:, 3] / Fan_height)

    # Weight calculation
    w = density_Al*(X[:, 3]*X[:, 2]*X[:, 4]+X[:, 7]*(X[:, 5]*X[:, 8]*X[:, 4]))+ Fan_Weight*N_fan

    return w

# Generate samples as before
samples = sobol_sample.sample(problem, N)

# Rejection sampling
valid_samples = np.array([s for s in samples if check_constraints(s)])

# Evaluate the weights
Y = evaluate_weight(valid_samples)

# Proceed with Sobol analysis as before
sobol_indices = sobol.analyze(problem, Y)
sobol_indices = sobol_indices[index_type]
variable_names = problem['names']

In [ ]:
# Extract and print first-order Sobol indices
indices_with_names = list(zip(sobol_indices, variable_names))
indices_with_names.sort(reverse=True)
sorted_indices, sorted_names = zip(*indices_with_names)

# Creating the plot
plt.figure(figsize=(10, 6))
plt.barh(range(len(sorted_indices)), sorted_indices, tick_label=sorted_names)
plt.xlabel('First-order Sobol Index')
plt.title('Feature Sensitivity Ranking: Weight')
plt.gca().invert_yaxis()  # Invert y-axis to have the most sensitive feature on top
plt.show()